# 버스 정류장 위치와 서울시 구/법정동 연결

### 1. 데이터 준비

In [449]:
# 1-1. 필요 라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [450]:
# 1-2. 데이터 파일 불러오기
# 법정동 경계파일 불러오기
dong_gdf = gpd.read_file\
    ('../4.Public_transportation/LSMD_ADM_SECT_UMD_Ldong/LSMD_ADM_SECT_UMD_11_202409.shp',
     encoding='cp949')

# 구 경계파일 불러오기
gu_gdf = gpd.read_file\
    ('../4.Public_transportation/LARD_ADM_SECT_SGG_gu/LARD_ADM_SECT_SGG_11_202405.shp',
     encoding='cp949')

# 구별 법정동 목록파일 불러오기
gu_dong_df = pd.read_csv\
    ('../4.Public_transportation/Gu_and_Legal_dong.csv', encoding='cp949')

# 버스 정류장역 데이터 불러오기
bus_df = pd.read_csv\
    ('../4.Public_transportation/Bus_stop_latitude_and_longitude.csv', encoding='cp949')

### 2. 불러온 데이터 확인

In [451]:
print("=== 데이터 현황 ===")
print("\n[구 경계 데이터]")
print("- 데이터 크기:", gu_gdf.shape)
print("- 컬럼 목록:", gu_gdf.columns.tolist())
print("- 구 목록:", sorted(gu_gdf['SGG_NM'].tolist()))
print("- 좌표계:", gu_gdf.crs)

print("\n[법정동 경계 데이터]")
print("- 데이터 크기:", dong_gdf.shape)
print("- 컬럼 목록:", dong_gdf.columns.tolist())
print("- 좌표계:", dong_gdf.crs)

print("\n[버스정류장 데이터]")
print("- 데이터 크기:", bus_df.shape)
print("- 컬럼 목록:", bus_df.columns.tolist())

print("\n[구별 법정동 데이터]")
print("- 데이터 크기:", gu_dong_df.shape)
print("- 컬럼 목록:", gu_dong_df.columns.tolist())

=== 데이터 현황 ===

[구 경계 데이터]
- 데이터 크기: (25, 5)
- 컬럼 목록: ['ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE', 'geometry']
- 구 목록: ['서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구']
- 좌표계: PROJCS["Korea_2000_Korea_Central_Belt_2010",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]



In [452]:
bus_df['ID'] = bus_df['ID'].astype(str)
gu_dong_df[gu_dong_df.duplicated(subset='legal_dong')]

,ID,gu,legal_dong
244,1168010700,강남구,신사동
251,1147010100,양천구,신정동


### 3. 데이터 전처리

In [453]:
# 3-1. 좌표계 통일 (모두 EPSG:4326으로 변환)
dong_gdf = dong_gdf.to_crs("EPSG:4326")
gu_gdf = gu_gdf.to_crs("EPSG:4326")

# 3-2. 버스 정류장 데이터를 GeoDataFrame으로 변환
bus_geometry = [Point(xy) for xy in zip(bus_df['longitude'], bus_df['latitude'])]
bus_gdf = gpd.GeoDataFrame(
    bus_df, 
    geometry=bus_geometry,
    crs="EPSG:4326"
)

print("\n=== 전처리 결과 ===")
print("- 법정동 좌표계:", dong_gdf.crs)
print("- 구 좌표계:", gu_gdf.crs)
print("- 버스 정류장 좌표계:", bus_gdf.crs)


=== 전처리 결과 ===
- 법정동 좌표계: EPSG:4326
- 구 좌표계: EPSG:4326
- 버스 정류장 좌표계: EPSG:4326


In [454]:
# 3-4. 컬럼 정리
gu_dong_df = gu_dong_df[['gu', 'legal_dong']]
gu_dong_df.columns = ['gu', 'dong']
gu_dong_df['gu_dong'] = gu_dong_df['gu'] + '_' + gu_dong_df['dong']
gu_dong_df.info(), gu_dong_df['gu'].nunique(), \
    gu_dong_df['dong'].nunique(), gu_dong_df['gu_dong'].nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gu       467 non-null    object
 1   dong     467 non-null    object
 2   gu_dong  467 non-null    object
dtypes: object(3)
memory usage: 11.1+ KB


(None, 25, 465, 467)

### 4. 공간 분석 작업

In [455]:
# 4-1 정류장과 동 매칭
buses_with_dong = gpd.sjoin(bus_gdf, dong_gdf, how='left', predicate='intersects')

# 4-3 최종 데이터프레임 생성
# 인덱스를 리셋하고 새로운 DataFrame 생성
result_df = pd.DataFrame({
    'Id': buses_with_dong['ID'].reset_index(drop=True),
    'bus_stop_name': buses_with_dong['bus_stop_name'].reset_index(drop=True),
    'dong': buses_with_dong['EMD_NM'].reset_index(drop=True)
})
result_df.info(), result_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11295 entries, 0 to 11294
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             11295 non-null  object
 1   bus_stop_name  11295 non-null  object
 2   dong           11278 non-null  object
dtypes: object(3)
memory usage: 264.9+ KB


(None,
           Id  bus_stop_name  dong
 0  100000001        종로2가사거리  종로2가
 1  100000002    창경궁.서울대학교병원   연건동
 2  100000003      명륜3가.성대입구  명륜4가
 3  100000004       종로2가.삼일교   관철동
 4  100000005  혜화동로터리.여운형활동터   혜화동)

In [456]:
# 4-5. 동이 NaN인 행 지우기
result_df = result_df.dropna(subset=['dong'])
result_df = result_df.dropna(subset=['Id'])
result_df = result_df.dropna(subset=['bus_stop_name'])

result_df.info(), result_df.head(), result_df.nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 11278 entries, 0 to 11294
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             11278 non-null  object
 1   bus_stop_name  11278 non-null  object
 2   dong           11278 non-null  object
dtypes: object(3)
memory usage: 352.4+ KB


(None,
           Id  bus_stop_name  dong
 0  100000001        종로2가사거리  종로2가
 1  100000002    창경궁.서울대학교병원   연건동
 2  100000003      명륜3가.성대입구  명륜4가
 3  100000004       종로2가.삼일교   관철동
 4  100000005  혜화동로터리.여운형활동터   혜화동,
 Id               11273
 bus_stop_name     7164
 dong               426
 dtype: int64)

In [457]:
# # 4-6. '서울특별시'와 '구' 이름 분리
# result_df['city'] = result_df['gu'].str.split(' ').str[0]
# result_df['gu'] = result_df['gu'].str.split(' ').str[1]
# result_df = result_df[['Id', 'bus_stop_name', 'gu', 'dong']]
# result_df.nunique()

# 4-7. 특정 열의 모든 공백 제거
result_df['dong'] = result_df['dong'].str.replace(' ', '')
result_df.nunique()

Id               11273
bus_stop_name     7164
dong               426
dtype: int64

In [458]:
# 4-8. 구별 법정동 목록과 버스 정류장 데이터 매칭
result_df = pd.merge(gu_dong_df, result_df, how='outer', on=['dong'])
result_df.nunique(), result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             11562 non-null  object
 1   dong           11562 non-null  object
 2   gu_dong        11562 non-null  object
 3   Id             11523 non-null  object
 4   bus_stop_name  11523 non-null  object
dtypes: object(5)
memory usage: 451.8+ KB


(gu                  25
 dong               465
 gu_dong            467
 Id               11273
 bus_stop_name     7164
 dtype: int64,
 None)

In [459]:
# 결과 확인
print("\n최종 데이터 정보:")
print(f"전체 행 수: {len(result_df)}개")
print(f"gu의 NaN 개수: {result_df['gu'].isna().sum()}개")
print(f"dong의 NaN 개수: {result_df['dong'].isna().sum()}개")
result_df.info(), result_df['gu'].nunique(), result_df['dong'].nunique(), \
    result_df['gu_dong'].nunique()


최종 데이터 정보:
전체 행 수: 11562개
gu의 NaN 개수: 0개
dong의 NaN 개수: 0개
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             11562 non-null  object
 1   dong           11562 non-null  object
 2   gu_dong        11562 non-null  object
 3   Id             11523 non-null  object
 4   bus_stop_name  11523 non-null  object
dtypes: object(5)
memory usage: 451.8+ KB


(None, 25, 465, 467)

### 5. 서울시 모든 구별 법정동과 매칭

In [460]:
# 6-1. 버스 정류장이 있는 법정동
yes_bus = result_df
print( yes_bus['gu_dong'].nunique() )

# 6-2. 버스 정류장이 없는 법정동
# 빈 데이터프레임 생성 (final_df와 동일한 컬럼을 사용)
no_bus = pd.DataFrame(columns=result_df.columns)
# bus_name 열이 NaN인 행 필터링
na_rows = result_df[result_df['bus_stop_name'].isna()]
# 필터링된 행들을 not_bus에 추가
no_bus = pd.concat([no_bus, na_rows], ignore_index=True)
no_bus.info()
print(no_bus['gu_dong'].nunique() )
print(no_bus['gu_dong'].unique() )

467
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             39 non-null     object
 1   dong           39 non-null     object
 2   gu_dong        39 non-null     object
 3   Id             0 non-null      object
 4   bus_stop_name  0 non-null      object
dtypes: object(5)
memory usage: 1.6+ KB
39
['종로구_궁정동' '중구_남산동3가' '중구_남학동' '종로구_내수동' '중구_다동' '종로구_당주동' '종로구_도렴동'
 '종로구_돈의동' '성북구_동선동3가' '중구_명동1가' '중구_명동2가' '중구_무교동' '중구_무학동' '성북구_보문동6가'
 '종로구_봉익동' '중구_북창동' '중구_산림동' '종로구_수송동' '중구_수하동' '종로구_신교동' '성북구_안암동3가'
 '영등포구_양평동' '중구_오장동' '중구_의주로2가' '종로구_익선동' '중구_인현동2가' '중구_장교동' '중구_저동2가'
 '종로구_적선동' '용산구_주성동' '중구_주자동' '종로구_청진동' '중구_초동' '중구_충무로3가' '중구_필동3가'
 '종로구_필운동' '서대문구_합동' '중구_회현동3가' '종로구_훈정동']


In [461]:
# 1. 두 데이터프레임의 'gu_dong' 값들을 비교
final_combinations = set(gu_dong_df['gu_dong'].unique())
result_combinations = set(result_df['gu_dong'].unique())

# 2. result_df에는 있지만 final_df에는 없는 조합 확인
diff_combinations = result_combinations - final_combinations
print("result_df에만 있는 구-동 조합 수:", len(diff_combinations))
print("\n차이나는 조합들:")
print(sorted(list(diff_combinations)))

# 3. 원본 데이터 확인
print("\nfinal_df의 gu, dong 컬럼 구조:")
print(result_df[['gu', 'dong']].head())
print("\nresult_df의 gu, dong 컬럼 구조:")
print(result_df[['gu', 'dong']].head())

# 4. 두 데이터프레임의 'gu_dong' 생성 방식 확인
print("\nfinal_df의 gu_dong 생성 방식:")
if 'gu_dong' in result_df.columns:
    print(result_df[['gu', 'dong', 'gu_dong']].head())

result_df에만 있는 구-동 조합 수: 0

차이나는 조합들:
[]

final_df의 gu, dong 컬럼 구조:
    gu dong
0  송파구  가락동
1  송파구  가락동
2  송파구  가락동
3  송파구  가락동
4  송파구  가락동

result_df의 gu, dong 컬럼 구조:
    gu dong
0  송파구  가락동
1  송파구  가락동
2  송파구  가락동
3  송파구  가락동
4  송파구  가락동

final_df의 gu_dong 생성 방식:
    gu dong  gu_dong
0  송파구  가락동  송파구_가락동
1  송파구  가락동  송파구_가락동
2  송파구  가락동  송파구_가락동
3  송파구  가락동  송파구_가락동
4  송파구  가락동  송파구_가락동


### 7. 결과 저장

In [462]:
# 버스 정류장역이 있는 법정동
yes_bus.to_csv('Data_preprocessing/seoul_bus_locations_yes.csv', index=False, encoding='cp949')

# 버스 정류장역이 없는 법정동
no_bus.to_csv('Data_preprocessing/seoul_bus_locations_no.csv', index=False, encoding='cp949')

# 둘을 종합한 법정동
result_df.to_csv('Data_preprocessing/seoul_bus_locations_all.csv', index=False, encoding='cp949')

In [463]:
# 상세 위치 정보 출력
print("\n=== 구별 버스 정류장역 상세 목록 ===")
for gu in sorted(result_df['gu'].unique()):
    print(f"\n[{gu}]")
    gu_data = result_df[result_df['gu'] == gu].sort_values(['dong', 'bus_stop_name'])
    for _, row in gu_data.iterrows():
        print(f"- {row['bus_stop_name']}({row['Id']}): {row['dong']}")


=== 구별 버스 정류장역 상세 목록 ===

[강남구]
- 강남구민체육관(122900087): 개포동
- 개원중학교.대모산입구역2번출구(122000256): 개포동
- 개일초등학교앞(122000238): 개포동
- 개포1단지(122000232): 개포동
- 개포1단지연금매장(122000240): 개포동
- 개포4단지5단지(122000358): 개포동
- 개포4단지5단지(122000372): 개포동
- 개포4단지6단지(122000359): 개포동
- 개포4단지6단지(122000371): 개포동
- 개포4단지7단지(122000360): 개포동
- 개포4단지7단지(122000385): 개포동
- 개포4복합문화센터.달터공원(122900003): 개포동
- 개포6단지앞(122000254): 개포동
- 개포고등학교(122000239): 개포동
- 개포고등학교앞(122000231): 개포동
- 개포도서관(122000242): 개포동
- 개포도서관(122000349): 개포동
- 개포동(122000626): 개포동
- 개포동역1번출구(122000362): 개포동
- 개포동역6번출구.개포시장(122000250): 개포동
- 개포동역7번출구.개포시장(122000248): 개포동
- 개포동역8번출구(122900053): 개포동
- 개포래미안포레스트(122000260): 개포동
- 개포시장.개포5단지상가(122000251): 개포동
- 개포시장.개포5단지상가(122000252): 개포동
- 개포우성6차아파트(122000326): 개포동
- 개포우성아파트(122000333): 개포동
- 개포자이프레지던스(122000246): 개포동
- 개포주공1단지아파트(122000345): 개포동
- 개포주공5단지경기여고(122000249): 개포동
- 개포주공7단지앞(122000257): 개포동
- 개포중학교(122000343): 개포동
- 개포지구대(122000274): 개포동
- 개포파크빌리지(122000234): 개포동
- 경기여고(122000247): 개포동
- 광수빌딩(1220002